# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

In [6]:
train_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.017717     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [7]:
example_weight_summary = example_model.coefficients
print (example_weight_summary)

+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None | 87910.07249239576  | 7873.338143401678  |
| sqft_living |  None | 315.4034405520999  | 3.455700325854737  |
|   bedrooms  |  None | -65080.21555282656 | 2717.4568544207104 |
|  bathrooms  |  None | 6944.020192637415  | 3923.1149314414843 |
+-------------+-------+--------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
example_predictions = example_model.predict(train_data)
print (example_predictions[0]) # should be 271789.505878

271789.50587803137


In [11]:
example_predictions*example_predictions

dtype: float
Rows: 17384
[73869535505.42445, 516788905118.0847, 43078791933.52973, 217726771927.4753, 190475724253.72098, 2459640008819.821, 201789958145.45325, 56364981527.416405, 212550108729.64468, 256179702864.51096, 1066811605220.7765, 109271006182.74384, 113517899515.74518, 120601497058.41101, 252677097596.3872, 115042214518.4327, 117770995715.31345, 86358252560.33783, 122986856292.70195, 763432028911.8351, 245852203693.80243, 93420304436.52446, 463804449662.0827, 459087405246.4562, 116402607006.8801, 147636297622.02792, 429727582312.2764, 117989341213.6906, 330201087387.9229, 246429195871.49338, 403806093978.6138, 128237716918.8045, 343196051187.31396, 48103643344.26137, 429727582312.2764, 838104397650.8081, 155864550948.6064, 92008997860.52274, 822112775516.5366, 44885868130.93455, 321742090128.3019, 89446311332.83116, 605805602854.1516, 75593950343.65338, 598131911047.8235, 122178429957.38066, 353311941343.6678, 408556739221.97546, 543966723030.35754, 453340703614.9814, 656323

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [15]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    prediction = model.predict(data)
    # Then compute the residuals/errors
    residual = outcome - prediction
    # Then square and add them up
    RSS = sum(residual**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [16]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print (rss_example_train) # should be 2.7376153833e+14

273761538330192.22


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [18]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [19]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)


In [21]:
# create the remaining 3 features in both TEST and TRAIN data

train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [29]:
import numpy as np

print(test_data['bedrooms_squared'].mean(), test_data['bed_bath_rooms'].mean(),
      test_data['log_sqft_living'].mean(), test_data['lat_plus_long'].mean())

12.4466777015843 7.5039016315913925 7.550274679645939 -74.65333497217311


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [31]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [32]:
# Learn the three models: (don't forget to set validation_set = None)
#eample_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                   # validation_set = None)

model_1 = turicreate.linear_regression.create(train_data, target='price', features=model_1_features,
                                             validation_set = None)
model_2 = turicreate.linear_regression.create(train_data, target='price', features=model_2_features,
                                             validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target='price', features=model_2_features,
                                             validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.016933     | 4014170.932934     | 235190.935428                   |

| 2         | 3        | 0.034033     | 4014170.932936     | 235190.935429                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [42]:
# Examine/extract each model's coefficients:

print(model_1.coefficients[model_1.coefficients['name'] == 'bathrooms'])
print(model_2.coefficients[model_2.coefficients['name'] == 'bathrooms'])



+-----------+-------+--------------------+--------------------+
|    name   | index |       value        |       stderr       |
+-----------+-------+--------------------+--------------------+
| bathrooms |  None | 13811.840541652908 | 3593.5421329711057 |
+-----------+-------+--------------------+--------------------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+-----------+-------+--------------------+-------------------+
|    name   | index |       value        |       stderr      |
+-----------+-------+--------------------+-------------------+
| bathrooms |  None | -77972.33051298813 | 7565.059910967307 |
+-----------+-------+--------------------+-------------------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [51]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
# def get_residual_sum_of_squares(model, data, outcome):

rss_model_1 = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

print(rss_model_1, rss_model_2, rss_model_3)
min(rss_model_1, rss_model_2, rss_model_3)

971328233545430.6 961592067857503.6 961592067857503.6


961592067857503.6

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [50]:
# Compute the RSS on TESTING data for each of the three models and record the values:

rss_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

print(rss_model_1_test, rss_model_2_test, rss_model_3_test)
min(rss_model_1_test, rss_model_2_test, rss_model_3_test)

226568089093161.2 224368799993971.03 224368799993971.03


224368799993971.03

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.